In [18]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
import glob
import os

CARGAR TODOS LOS ARCHIVOS Y CONCATENARLOS TODOS

In [4]:
#Hacer una lista con los archivos de la carpeta
archivos = glob.glob("/Users/nacho/Desktop/TFM/Datos/Datos equipos LaLiga J27/*xlsx")
len(archivos)

20

In [5]:
#Cargar los archivos
len(archivos)

lista_de_archivos = []
for f in archivos:
    data = pd.read_excel(f)
    nombre = os.path.basename(f)
    data['carpeta_contenedora'] = nombre
    lista_de_archivos.append(data)

In [6]:
len(lista_de_archivos)

20

In [7]:
#Concatenarlos en un solo data frame
df = pd.concat(lista_de_archivos, ignore_index= True)

In [8]:
df.shape

(1396, 105)

LIMPIAR Y ORDENAR LOS DATOS DEL ARCHIVO CONCATENADO

In [9]:
#Eliminar filas de cada partido, solo nos quedamos con las generales, las estadísticas de lo equipos y las de los rivales contra cada equipo.
df = df[~(df['Team'].str.startswith('2022') | df['Team'].str.startswith('2023'))]

In [10]:
#Crear dos df distientos uno con las estadísticas de lo equipos y otros con las estadísticas de los rivales contra cada equipo.
Opponents_stats = df[df['Team'].str.startswith('Opponents')]
Team_stats = df[~df['Team'].str.startswith('Opponents')]

3 FASE DEFENSIVA

3.1 Recuperaciones

In [11]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Total recoveries', ascending=True)

# Crear el gráfico
recuperaciones_totales = Team_stats.hvplot.barh(x='Team', y='Total recoveries',
                       color='Total recoveries', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Recuperaciones', 
                       title='Total recuperaciones por 90')

recuperaciones_totales

:Bars   [Team]   (Total recoveries)

In [12]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Recoveries low', ascending=True)

# Crear el gráfico
recuperaciones_bloque_bajo = Team_stats.hvplot.barh(x='Team', y='Recoveries low',
                       color='Recoveries low', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Recuperaciones en bloque bajo', 
                       title='Recuperaciones en bloque bajo por 90')

recuperaciones_bloque_bajo

:Bars   [Team]   (Recoveries low)

In [13]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Recoveries  medium', ascending=True)

# Crear el gráfico
recuperaciones_bloque_medio = Team_stats.hvplot.barh(x='Team', y='Recoveries  medium',
                       color='Recoveries  medium', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Recuperaciones en bloque medio', 
                       title='Recuperaciones en bloque medio por 90')

recuperaciones_bloque_medio

:Bars   [Team]   (Recoveries  medium)

In [14]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Recoveries high', ascending=True)

# Crear el gráfico
recuperaciones_bloque_alto = Team_stats.hvplot.barh(x='Team', y='Recoveries high',
                       color='Recoveries high', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Recuperaciones en bloque alto', 
                       title='Recuperaciones en bloque alto por 90')

recuperaciones_bloque_alto

:Bars   [Team]   (Recoveries high)

In [15]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('PPDA', ascending=False)

# Crear el gráfico
PPDA = Team_stats.hvplot.barh(x='Team', y='PPDA',
                       color='PPDA', cmap='RdYlGn_r', 
                       xlabel='Equipo', ylabel='PPDA', 
                       title='Pases permitidos por acción defensiva por 90')

PPDA

:Bars   [Team]   (PPDA)

3.2 Duelos

In [19]:
#Renombrar columna con error
Team_stats = Team_stats.rename(columns={'Total duels ': 'Total duels'})

In [20]:
# Calcular las medias de las columnas.
duelos_mean = Team_stats['Total duels'].mean()
duelos_ganados_mean = Team_stats['% of duels won'].mean()

# Crear una nueva columna de colores en el DataFrame
Team_stats['color'] = ['blue' if team == 'Espanyol' else 'grey' for team in Team_stats['Team']]

# Crear el gráfico con los colores específicos para cada equipo
duelos = Team_stats.hvplot.scatter(x='Total duels', y='% of duels won', by='Team', 
                                 legend='right', height=600, width=900, color='color', 
                                 hover_cols=["Total duels", "% of duels won"])

#Añadir lineas de media de goles y tiros para ver cuadrantes
duelos_mean_line = hv.VLine(duelos_mean)
duelos_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

duelos_ganados_mean_line = hv.HLine(duelos_ganados_mean)
duelos_ganados_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

duelos = duelos * duelos_ganados_mean_line * duelos_mean_line.opts(title='Duelos')

duelos

:Overlay
   .NdOverlay.I :NdOverlay   [Team]
      :Scatter   [Total duels]   (% of duels won,color)
   .HLine.I     :HLine   [x,y]
   .VLine.I     :VLine   [x,y]

In [21]:
# Calcular las medias de las columnas.
duelos_defensivos_mean = Team_stats['Defensive duels'].mean()
duelos_defensivos_ganados_mean = Team_stats['% of Defensive duels won'].mean()

# Crear una nueva columna de colores en el DataFrame
Team_stats['color'] = ['blue' if team == 'Espanyol' else 'grey' for team in Team_stats['Team']]

# Crear el gráfico con los colores específicos para cada equipo
duelos_defensivos = Team_stats.hvplot.scatter(x='Defensive duels', y='% of Defensive duels won', by='Team', 
                                 legend='right', height=600, width=900, color='color', 
                                 hover_cols=["Defensive duels", "% of Defensive duels won"])

#Añadir lineas de media de goles y tiros para ver cuadrantes
duelos_defensivos_mean_line = hv.VLine(duelos_defensivos_mean)
duelos_defensivos_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

duelos_defensivos_ganados_mean_line = hv.HLine(duelos_defensivos_ganados_mean)
duelos_defensivos_ganados_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

duelos_defensivos = duelos_defensivos * duelos_defensivos_mean_line * duelos_defensivos_ganados_mean_line.opts(title='Duelos defensivos')

duelos_defensivos

:Overlay
   .NdOverlay.I :NdOverlay   [Team]
      :Scatter   [Defensive duels]   (% of Defensive duels won,color)
   .VLine.I     :VLine   [x,y]
   .HLine.I     :HLine   [x,y]

In [22]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Sliding tackles', ascending=True)

# Crear el gráfico
tackles = Team_stats.hvplot.barh(x='Team', y='Sliding tackles',
                       color='Sliding tackles', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Tackles', 
                       title='Tackles por 90')

tackles

:Bars   [Team]   (Sliding tackles)

In [23]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Interceptions', ascending=True)

# Crear el gráfico
intercepciones = Team_stats.hvplot.barh(x='Team', y='Interceptions',
                       color='Interceptions', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Intercepciones', 
                       title='Intercepciones por 90')

intercepciones

:Bars   [Team]   (Interceptions)

In [24]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Clearances', ascending=True)

# Crear el gráfico
despejes = Team_stats.hvplot.barh(x='Team', y='Clearances',
                       color='Clearances', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Despeje', 
                       title='Despeje por 90')

despejes

:Bars   [Team]   (Clearances)

3.3 Rendimiento / concesiones

In [25]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Conceded goals', ascending=False)

# Crear el gráfico
goles_contra = Team_stats.hvplot.barh(x='Team', y='Conceded goals',
                       color='Conceded goals', cmap='RdYlGn_r', 
                       xlabel='Equipo', ylabel='Goles en contra', 
                       title='Goles en contra por 90')

goles_contra

:Bars   [Team]   (Conceded goals)

In [26]:
# Ordenar los equipos.
Opponents_stats = Opponents_stats.sort_values('xG', ascending=False)

# Crear el gráfico
xg_concedido = Opponents_stats.hvplot.barh(x='Team', y='xG',
                       color='xG', cmap='RdYlGn_r', 
                       xlabel='Equipo', ylabel='xG concedido', 
                       title='xG concedido por 90')

xg_concedido

:Bars   [Team]   (xG)

In [27]:
# Crear una nueva columna que mida la eficacia goleadora. Para comparar la calidad de las acciones y los goles.
Opponents_stats['Eficacia defensiva (xG - Goles)'] = round(Opponents_stats['xG'] - Opponents_stats['Goals'] , 5)

# Ordenar los equipos por xG.
Opponents_stats = Opponents_stats.sort_values('Eficacia defensiva (xG - Goles)', ascending=True)

# Crear el gráfico
eficacia_defensa = Opponents_stats.hvplot.barh(x='Team', y='Eficacia defensiva (xG - Goles)',
                       color='Eficacia defensiva (xG - Goles)', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Eficacia defensiva (xG - Goles)', 
                       title='Eficacia defensiva por 90',
                       hover_cols=["Goals", "xG"])

eficacia_defensa

:Bars   [Team]   (Eficacia defensiva (xG - Goles),Goals,xG)

In [28]:
# Calcular las medias de las columnas.
tiros_rivales_mean = Team_stats['Shots against'].mean()
tiros_rivales_puerta_mean = Team_stats['% of shots against on target'].mean()

# Crear una nueva columna de colores en el DataFrame
Team_stats['color'] = ['blue' if team == 'Espanyol' else 'grey' for team in Team_stats['Team']]

# Crear el gráfico con los colores específicos para cada equipo
tiros_rivales = Team_stats.hvplot.scatter(x='Shots against', y='% of shots against on target', by='Team', 
                                 legend='right', height=600, width=900, color='color', 
                                 hover_cols=["Shots against", "% of shots against on target"])

#Añadir lineas de media de goles y tiros para ver cuadrantes
tiros_rivales_mean_line = hv.VLine(tiros_rivales_mean)
tiros_rivales_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

tiros_rivales_puerta_mean_line = hv.HLine(tiros_rivales_puerta_mean)
tiros_rivales_puerta_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

tiros_rivales = tiros_rivales * tiros_rivales_mean_line * tiros_rivales_puerta_mean_line.opts(title='Tiros concedidos')

tiros_rivales

:Overlay
   .NdOverlay.I :NdOverlay   [Team]
      :Scatter   [Shots against]   (% of shots against on target,color)
   .VLine.I     :VLine   [x,y]
   .HLine.I     :HLine   [x,y]

In [29]:
# Ordenar los equipos por xG.
Opponents_stats = Opponents_stats.sort_values('Deep completed passes', ascending=False)

# Crear el gráfico
pases_profundidad_concedidos = Opponents_stats.hvplot.barh(x='Team', y='Deep completed passes',
                       color='Deep completed passes', cmap='RdYlGn_r', 
                       xlabel='Equipo', ylabel='Deep completed passes', 
                       title='Pases en profundidad concedidos por 90')

pases_profundidad_concedidos

:Bars   [Team]   (Deep completed passes)

In [30]:
# Ordenar los equipos.
Opponents_stats = Opponents_stats.sort_values('Penalty area entries', ascending=False)

# Crear el gráfico
entradas_area_concedidas = Opponents_stats.hvplot.barh(x='Team', y='Penalty area entries',
                       color='Penalty area entries', cmap='RdYlGn_r', 
                       xlabel='Equipo', ylabel='Penalty area entries', 
                       title='Entradas al área permitidas por 90')

entradas_area_concedidas

:Bars   [Team]   (Penalty area entries)

In [31]:
Team_stats['Total cards'] = Team_stats['Yellow cards'] + Team_stats['Red cards']

#Calcular las medias de las columnas de centros y pases profundos.
cards_mean = Team_stats['Total cards'].mean()
fouls_mean = Team_stats['Fouls'].mean()

#Crear el gráfico con los colores específicos para cada equipo
fouls_cards = Team_stats.hvplot.scatter(x='Total cards', y='Fouls', by='Team', 
                                 legend='right', height=600, width=900, color='color', 
                                 hover_cols=["Total cards", "Fouls"])

#Añadir lineas de media de goles y tiros para ver cuadrantes
cards_mean_line = hv.VLine(cards_mean)
cards_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

fouls_mean_line = hv.HLine(fouls_mean)
fouls_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

fouls_cards = fouls_cards * cards_mean_line * fouls_mean_line.opts(title='Relación faltas / tarjetas')

fouls_cards

:Overlay
   .NdOverlay.I :NdOverlay   [Team]
      :Scatter   [Total cards]   (Fouls,color)
   .VLine.I     :VLine   [x,y]
   .HLine.I     :HLine   [x,y]

Guardar plots f. defensiva

In [32]:
#Guardar visualizaciones. Nombre = fase ofesniva(3)_subfase(1,2,3...)_visualización

#1. Recuperaciones
hvplot.save(recuperaciones_totales, '3_1_recuperaciones_totales.png')
hvplot.save(recuperaciones_bloque_bajo, '3_1_recuperaciones_bloque_bajo.png')
hvplot.save(recuperaciones_bloque_medio, '3_1_recuperaciones_bloque_medio.png')
hvplot.save(recuperaciones_bloque_alto, '3_1_recuperaciones_bloque_alto.png')
hvplot.save(PPDA, '3_1_PPDA.png')

#2. Duelos
hvplot.save(duelos, '3_2_duelos.png')
hvplot.save(duelos_defensivos, '3_2_duelos_defensivos.png')
hvplot.save(tackles, '3_2_tackles.png')
hvplot.save(intercepciones, '3_2_intercepciones.png')
hvplot.save(despejes, '3_2_despejes.png')

#3. Rendimiento / concesiones
hvplot.save(goles_contra, '3_3_goles_contra.png')
hvplot.save(xg_concedido, '3_3_xg_concedido.png') # calidad de las acciones concedidas
hvplot.save(eficacia_defensa, '3_3_eficacia_defensa.png') # desempeño de la defensa
hvplot.save(tiros_rivales, '3_3_tiros_rivales.png')
hvplot.save(pases_profundidad_concedidos, '3_3_pases_profundidad_concedidos.png')
hvplot.save(entradas_area_concedidas, '3_3_entradas_area_concedidas.png')
hvplot.save(fouls_cards, '3_3_fouls_cards.png')
hvplot.save(fouls_cards, '3_3_fouls_cards.png')